# Data cleaning, merging, and geocoding


## Setup


### Import libraries


In [ ]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
from haversine import Unit, haversine

from functions.data_etl.cleaning import convert_area_to_sqm
from functions.data_etl.geocoding import extract_geocoded_data_from_txt, geocode_addresses
from functions.data_etl.imputation import PowerCapacityScenario, impute_missing_values
from functions.data_etl.merging import generate_fuzzy_matches
from functions.project_settings import WGS84_CRS


## Setting input and output paths


In [9]:
COUNTRY_BOUNDARIES_INPUT = Path("data/inputs/common/country_boundaries/ne_110m_admin_0_countries.shp")
DATACENTERS_COM_INPUT = Path("data/outputs/0_webscraping/datacenters_com.csv")
DATACENTERMAP_INPUT = Path("data/outputs/0_webscraping/datacentermap.com.xlsx")

OUTPUT_DIR = Path("data/outputs/1_data_etl/")
GEOCODING_API_RESULTS_PATH = OUTPUT_DIR / "geocoding_results.txt"
DATACENTERMAP_GEOCODED_OUTPUT = OUTPUT_DIR / "datacentermap_geocoded.csv"

## Manual edits
# For adding missing country codes
DATACENTERMAP_GDF_PRE_MANUAL_EDIT_1 = OUTPUT_DIR / "datacentermap_v1_pre_edit_gdf.csv"
DATACENTERS_COM_GDF_PRE_MANUAL_EDIT_1 = OUTPUT_DIR / "datacenters_com_v1_pre_edit_gdf.csv"
DATACENTERMAP_GDF_POST_MANUAL_EDIT_1 = OUTPUT_DIR / "datacentermap_v1_post_edit_gdf.csv"
DATACENTERS_COM_GDF_POST_MANUAL_EDIT_1 = OUTPUT_DIR / "datacenters_com_v1_post_edit_gdf.csv"

# For fixing duplicate datacentermap entries
DATACENTERMAP_GDF_DUPLICATES_PRE_MANUAL_EDIT = OUTPUT_DIR / "datacentermap_v2_pre_edit_gdf.csv"
DATACENTERMAP_GDF_DUPLICATES_POST_MANUAL_EDIT = OUTPUT_DIR / "datacentermap_v2_post_edit_gdf.csv"

# For fuzzy matching manual inspection
FUZZY_MATCHES_PRE_MANUAL_EDIT = OUTPUT_DIR / "fuzzy_matches_pre_manual_edit.csv"
FUZZY_MATCHES_POST_MANUAL_EDIT = OUTPUT_DIR / "fuzzy_matches_post_manual_edit.csv"

# For manual data collection for Amazon, Google, Meta, Microsoft, and Apple
DATA_CENTERS_MISSING_SPECS_PRE_MANUAL_EDIT = OUTPUT_DIR / "data_centers_missing_specs_pre_edit.csv"
DATA_CENTERS_MISSING_SPECS_POST_MANUAL_EDIT = OUTPUT_DIR / "data_centers_missing_specs_post_edit.csv"
DATA_CENTERS_WITH_SPECS_PRE_MANUAL_EDIT = OUTPUT_DIR / "data_centers_with_specs_pre_edit.csv"
DATA_CENTERS_WITH_SPECS_POST_MANUAL_EDIT = OUTPUT_DIR / "data_centers_with_specs_post_edit.csv"

## Final output prefix (will have min, max, avg suffixes)
DATACENTERS_FINAL_OUTPUT_PREFIX = OUTPUT_DIR / "data_centers_"


## Geocoding

The web scraped datacenters.com information has latitude and longitude provided, so we only geocode datacentermap.com.


In [ ]:
# Import the datacentermap.com data
datacentermap = pd.read_excel(
    DATACENTERMAP_INPUT,
    usecols=["company", "name", "address", "total_space", "white_space", "critical_power_mw", "notes"],
)

# Extract the addresses
addresses_datacentermap = datacentermap["address"].tolist()

In [ ]:
# Geocode the addresses and store the results in a text file
google_maps_api_key = input("Please enter your Google Maps API key: ")
geocode_addresses(addresses_datacentermap, google_maps_api_key, GEOCODING_API_RESULTS_PATH)

### Post processing

Geocoding is expensive and time intensive, so we save results to a text file in the case we have to do multiple rounds of geocoding.


In [ ]:
# Extract the address, latitude, and longitude from the text file
geocoded_info = extract_geocoded_data_from_txt(GEOCODING_API_RESULTS_PATH)

In [ ]:
# Delete the address column from the datacentermap data
datacentermap = datacentermap.drop(columns=["address"])

# Append the geocoded information to the datacentermap dataframe
datacentermap_geocoded = pd.concat(
    [datacentermap, geocoded_info], axis=1
)  # Ideally we would merge on the ID instead of concatenating

In [ ]:
# Write the results to a CSV file
datacentermap_geocoded.to_csv(DATACENTERMAP_GEOCODED_OUTPUT, index=False)

## Data cleaning


### datacenters.com


In [10]:
# Import the data
datacenters_com = pd.read_csv(DATACENTERS_COM_INPUT)

In [ ]:
# Remove duplicates
datacenters_com = datacenters_com.drop_duplicates(subset=["company", "name", "address"])

# Convert the area columns from square feet to square meters
for col_name in ["total_space_sqft", "colocation_space_sqft"]:
    datacenters_com[col_name.replace("sqft", "m2")] = (
        datacenters_com[col_name].str.replace(",", "").astype(float) * 0.092903  # 1 sqft = 0.092903 m2
    )
    datacenters_com = datacenters_com.drop(columns=[col_name])

### datacentermap.com


In [ ]:
# Read the geocoded datacentermap data
datacentermap = pd.read_csv(DATACENTERMAP_GEOCODED_OUTPUT)

In [ ]:
# Convert the total and white space to square meters
datacentermap["total_space_m2"] = datacentermap["total_space"].apply(convert_area_to_sqm)
datacentermap["white_space_m2"] = datacentermap["white_space"].apply(convert_area_to_sqm)

# A data center is considered operational if it does not have the word "planned" in the notes
datacentermap["operational"] = ~datacentermap["notes"].fillna("").str.contains("planned", case=False)

# Drop unnecessary columns
datacentermap = datacentermap.drop(columns=["total_space", "white_space", "notes"])

## Merging

Datacentermap has greater accuracy (based on web searches on data center websites), so we remove duplicate data centers from the datacenters_com.


In [ ]:
# Exclude data centers without the required information
datacenters_com = datacenters_com.dropna(subset=["total_space_m2", "white_space_m2", "critical_power_mw"], how="all")

# Exclude data_centers.com entries that appear in the datacentermap data
datacenters_com = datacenters_com[~datacenters_com["name"].str.lower().isin(datacentermap["name"].str.lower())]

In [ ]:
# Creating GeoDataFrames for the data sets
datacenters_com_gdf, datacentermap_gdf = (
    gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=WGS84_CRS)
    for df in (datacenters_com, datacentermap)
)

In [ ]:
# Load world map with country borders
country_boundaries = gpd.read_file(COUNTRY_BOUNDARIES_INPUT)

# Add country iso code column to the datacenter GeoDataFrames
datacenters_com_gdf, datacentermap_gdf = (
    gpd.sjoin(df, country_boundaries[["geometry", "ISO_A3"]], how="left", predicate="within")
    for df in (datacenters_com_gdf, datacentermap_gdf)
)

The country classifications are missing for about 150 data centers, so we export the data, add them manually, and re-import the data.


In [ ]:
# Export the dataframes to csv for manual edits
datacenters_com_gdf.to_csv(DATACENTERS_COM_GDF_PRE_MANUAL_EDIT_1, index=False)
datacentermap_gdf.to_csv(DATACENTERMAP_GDF_PRE_MANUAL_EDIT_1, index=False)

In [ ]:
# Reimport the dataframes after manual edits
datacenters_com_gdf = pd.read_csv(DATACENTERS_COM_GDF_POST_MANUAL_EDIT_1)
datacentermap_gdf = pd.read_csv(DATACENTERMAP_GDF_POST_MANUAL_EDIT_1)

During manual inspection, it was observed that due to the manual data collection of datacentermap that some data centers occurred twice, with slight differences in syntax (for instance extra spaces in data center names). Hence, these need to be removed manually, as the data was inconsistent across duplicates


In [ ]:
# Normalize the datacenter names
datacentermap_gdf["name_stripped"] = datacentermap_gdf["name"].str.lower().str.replace(r"\W+", "", regex=True)

# Create a dataframe with the duplicate names for manual inspection
duplicate_names_datacentermap = datacentermap_gdf[datacentermap_gdf.duplicated(subset="name_stripped", keep=False)]

# Keep the rest of the data without these duplicates
datacentermap_without_name_duplicates = datacentermap_gdf.drop_duplicates(subset="name_stripped", keep=False)

In [ ]:
# Export the duplicate names for manual edits
duplicate_names_datacentermap.to_csv(DATACENTERMAP_GDF_DUPLICATES_PRE_MANUAL_EDIT, index=False)

Manual inspection took place in Excel, and the data was saved as a CSV file.


In [ ]:
# Re-import the corrected duplicates
duplicate_names_datacentermap_corrected = pd.read_csv(DATACENTERMAP_GDF_DUPLICATES_POST_MANUAL_EDIT)

In [ ]:
# Merge the corrected duplicates with the rest of the data and remove the stripped name column
datacentermap_gdf = pd.concat([datacentermap_without_name_duplicates, duplicate_names_datacentermap_corrected]).drop(
    columns=["name_stripped"]
)

### Fuzzy matching


In [ ]:
# Drop duplicates that accrued during multiple geocoding attempts
datacenters_com_gdf, datacentermap_gdf = (
    df.drop_duplicates(subset=df.columns.difference(["latitude", "longitude", "geometry"]))
    for df in (datacenters_com_gdf, datacentermap_gdf)
)

In [ ]:
# Perform fuzzy matching, pre-filtering the data by country to reduce the number of comparisons
fuzzy_matches = generate_fuzzy_matches(
    datacentermap_gdf, datacenters_com_gdf, match_columns=["company", "name"], country_col="ISO_A3", threshold=30
)

In [ ]:
# Calculate the distance for each matching pair
fuzzy_matches["distance"] = fuzzy_matches.apply(
    lambda x: haversine(
        (x["latitude_df1"], x["longitude_df1"]), (x["latitude_df2"], x["longitude_df2"]), unit=Unit.METERS
    ),
    axis=1,
)

# Remove pairs further than 1150 meters apart. This was found to be a good threshold by manual inspection of the data
fuzzy_matches = fuzzy_matches[fuzzy_matches["distance"] < 1150]

In [ ]:
# Take a subset of the matches for manual inspection
subset_columns = ["company_df1", "name_df1", "address_df1", "company_df2", "name_df2", "address_df2", "best_score"]
fuzzy_matches_manual_edits = fuzzy_matches[subset_columns]

In [ ]:
# Export the dataframe to csv for manual edits
fuzzy_matches_manual_edits.to_csv(FUZZY_MATCHES_PRE_MANUAL_EDIT)

Inspect the fuzzy matches and manually remove those which are not actually matches.


In [ ]:
# Reimport the dataframe after manual edits
fuzzy_matches_manual_edits = pd.read_csv(FUZZY_MATCHES_POST_MANUAL_EDIT)

In [ ]:
# Filter fuzzy matches to keep only those from after the manual inspection
fuzzy_matches = fuzzy_matches.merge(fuzzy_matches_manual_edits, on=subset_columns, how="inner")

Now that the fuzzy matching is complete, we update the missing specifications for data centers from datacentermap.com with the specifications from their match in datacenters.com.


In [ ]:
# Updating specifications for total space, white space, and critical power
fuzzy_matches = fuzzy_matches.fillna(
    {
        "total_space_m2_df1": fuzzy_matches["total_space_m2_df2"],
        "white_space_m2_df1": fuzzy_matches["white_space_m2_df2"],
        "critical_power_mw_df1": fuzzy_matches["critical_power_mw_df2"],
    }
)

In [ ]:
# Remove the entries from datacenters.com that also occur in datacentermap.com, based on company, name, and address
datacenters_com_key = datacenters_com_gdf[["company", "name", "address"]].agg("-".join, axis=1)
fuzzy_matches_key = fuzzy_matches[["company_df2", "name_df2", "address_df2"]].agg("-".join, axis=1)

datacenters_com_gdf = datacenters_com_gdf[~datacenters_com_key.isin(fuzzy_matches_key)]

In [ ]:
# Drop duplicates in datacentermap.com with the exception of the company column
datacentermap_gdf = datacentermap_gdf.drop_duplicates(subset=datacentermap_gdf.columns.difference(["company"]))

Datacentermap data is then updated based on the data filled in from datacenters.com for the matches


In [ ]:
# Merge and update values in one go
merged_df = datacentermap_gdf.merge(
    fuzzy_matches[
        ["company_df1", "name_df1", "address_df1", "total_space_m2_df1", "white_space_m2_df1", "critical_power_mw_df1"]
    ],
    left_on=["company", "name", "address"],
    right_on=["company_df1", "name_df1", "address_df1"],
    how="left",
)

# Update the three specification columns
for original, update in [
    ("total_space_m2", "total_space_m2_df1"),
    ("white_space_m2", "white_space_m2_df1"),
    ("critical_power_mw", "critical_power_mw_df1"),
]:
    merged_df[original] = merged_df[update].combine_first(merged_df[original])

# Keep only original columns
datacentermap_gdf = merged_df[datacentermap_gdf.columns]

In [ ]:
# Add operational status to all datacenters_com entries
datacenters_com_gdf["operational"] = True

# Add a new column which identifies which data source the data comes from
datacenters_com_gdf["data_source"] = "datacenters_com"
datacentermap_gdf["data_source"] = "datacentermap"

In [ ]:
# Merge the databases
data_centers = pd.concat([datacenters_com_gdf, datacentermap_gdf])

## Filling in missing specs from Amazon, Meta, Google, Apple, and Microsoft

These large tech companies do not report data center information such as area or critical power despite being major players in the data center market. Therefore, manual data collection from newspapers and data center blogs on the total data center space was performed. This resulted in data for about 1/3rd of the data centers in question.


In [ ]:
BIG_5_COMPANIES = ["Amazon AWS", "Meta", "Google", "Apple", "Microsoft"]
AMAZON_AWS = "Amazon AWS"

In [ ]:
# Extract data centers without total space, nor white space, nor critical power specs
data_centers_missing_specs = data_centers[
    data_centers[["total_space_m2", "white_space_m2", "critical_power_mw"]].isna().all(axis=1)
]

# Separate data centers with specs
data_centers_with_specs = data_centers.drop(data_centers_missing_specs.index)

In [ ]:
# Export to csv for manual data additions
data_centers_missing_specs.to_csv(DATA_CENTERS_MISSING_SPECS_PRE_MANUAL_EDIT, index=False)
data_centers_with_specs.to_csv(DATA_CENTERS_WITH_SPECS_PRE_MANUAL_EDIT, index=False)

During further inspection at this stage, it was observed that fuzzy matching was insufficient to catch all matches between the two data sources. Therefore, manual removal of the matches from datacenters.com (while replacing missing data from datacentermap) was performed.

For adding information for Apple, Amazon, Meta, and Google which were missing information, internet searches were performed for each of the approximately 300 data centers. Square footage, often from news articles, was collected and added with the source reported in the "source link" column.


In [ ]:
# Import after manual data additions
data_centers_missing_specs = pd.read_csv(DATA_CENTERS_MISSING_SPECS_POST_MANUAL_EDIT)
data_centers_with_specs = pd.read_csv(DATA_CENTERS_WITH_SPECS_POST_MANUAL_EDIT)

In [ ]:
# Merge the data centers with and without specs
data_centers = pd.concat([data_centers_with_specs, data_centers_missing_specs])

In [ ]:
# Drop any remaining duplicates based on 'company', 'name, 'address'
data_centers = data_centers.drop_duplicates(subset=["company", "name", "address"])

# Convert any remaining strings to floats in the total_space_m2, white_space_m2, critical_power_mw columns
columns_to_convert = ["total_space_m2", "white_space_m2", "critical_power_mw"]
data_centers[columns_to_convert] = data_centers[columns_to_convert].apply(pd.to_numeric, errors="coerce")

### Final data cleaning


Upon inspecting the data, some critical power figures from datacenters.com are inconceivable large, even when assumed to be reported in kW and converted to MW. Therefore, for critical power ratings above 300 MW from datacenters.com, we remove this statistic.


In [ ]:
CRITICAL_POWER_UPPER_LIMIT = 300
DATACENTERS_COM_STR = "datacenters_com"

# Remove critical power outlier values
data_centers.loc[
    (data_centers["critical_power_mw"] > CRITICAL_POWER_UPPER_LIMIT)
    & (data_centers["data_source"] == DATACENTERS_COM_STR),
    "critical_power_mw",
] = np.nan

# Change remaining floats with 0 values to nan
for column in ["total_space_m2", "white_space_m2", "critical_power_mw"]:
    data_centers.loc[data_centers[column] == 0.0, column] = np.nan

In [ ]:
# Remove data centers with no specs, except when company name is one of the five to be imputed
data_centers = data_centers[
    data_centers["company"].isin(BIG_5_COMPANIES)
    | data_centers[["total_space_m2", "white_space_m2", "critical_power_mw"]].notna().any(axis=1)
]

In [ ]:
# Export to csv
data_centers.to_csv(f"{DATACENTERS_FINAL_OUTPUT_PREFIX}impute_baseline.csv", index=False)

### Data imputation

For the Big 5 companies, many data centers are still missing information. In order to obtain a more complete picture of data center impacts, a minimum and maximum estimate of floor space is filled in for the locations with missing information.


In [ ]:
# Create base dataframes for min, max, and avg estimates
total_space_estimates = {scenario: data_centers.copy() for scenario in PowerCapacityScenario}

# Companies to process in order
companies = BIG_5_COMPANIES

# Fill missing total space for each company and aggregation method
for scenario, df in total_space_estimates.items():
    for company in companies:
        result = impute_missing_values(
            df=df,
            company_name=company,
            power_capacity_scenario=scenario,
            # Amazon listings can be individual buildings or campuses with multiple data centers.
            # We exclude the word "Campus" from the name to avoid overestimating the total space.
            name_should_not_contain="Campus" if company == AMAZON_AWS else None,
            target_column="total_space_m2",
        )
        total_space_estimates[scenario] = result

# Remove NaN values and export
for scenario, df in total_space_estimates.items():
    df_cleaned = df.dropna(subset=["total_space_m2", "white_space_m2", "critical_power_mw"], how="all")
    df_cleaned.to_csv(f"{DATACENTERS_FINAL_OUTPUT_PREFIX}impute_{scenario}.csv", index=False)